In [ ]:
import pandas as pd
import numpy as np

import os

from load_wastewater_data import *
from plotting_tools import *
import seaborn as sns

# Update rcParams to set the default font to Times New Roman
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'DejaVu Serif'

In [ ]:
# define experiment to be preprocessed
experiment_series = "2024_09_17"

resolution = "3_min_resolution"
rain_scenario = "KeinRegen" # one of KeinRegen, Nieselregen, MittelstarkerRegen
degradation_setting = "no_decay" # one of "no_decay", "linear_decay_dynamics", "constant_decay_dynamics"

In [ ]:
df = load_systems_data(experiment_series, resolution, rain_scenario, degradation_setting, file_type="concentrations")

In [ ]:
plot_path = f"../plots/sampling_strategies"
os.makedirs(os.path.join(plot_path), exist_ok=True)
file_name_prefix = f"{resolution}_{rain_scenario}_{degradation_setting}"

In [ ]:
df["time"] = pd.Timestamp("2024-01-01") + pd.to_timedelta(df["minutes"], unit="minutes")

In [ ]:
df.time.min(), df.time.max()

## Preprocessing

In [ ]:
def simulate_sampling_strategies(df, resolution):
    df.time = pd.to_datetime(df.time)
    df["day"]= [el.day for el in df.time]
    df["hour"] = [el.hour for el in df.time]

    # resample every 3 minutes and fill with the last available value
    resampled_df = df.set_index("time").groupby(["sampling_point", "memilio_id"]).resample(f"{resolution}min").ffill()
    # get hourly measurements
    resampled_df = resampled_df.loc[resampled_df.minutes.mod(60) == 0,:]
    # fix column naming and index
    resampled_df.index = [el[2] for el in resampled_df.index]
    resampled_df = resampled_df.reset_index().rename(columns={"index": "time"})
    # calculate 24h samples
    df_24h = resampled_df.groupby(["sampling_point", "memilio_id", "day"]).mean().reset_index().rename(columns={"COVID_copies/l": "COVID_24h_sample", "PMMoV_copies/l": "PMMoV_24h_sample"})
    # only consider 24h samples with 24h of data
    df_24h = df_24h.loc[df_24h.hour==11.5, ["time", "time_in_days", "sampling_point", "memilio_id", "COVID_24h_sample", "PMMoV_24h_sample"]]
    df_24h["time_in_days"] = df_24h["time_in_days"] + 1.5/24 # plot at 14:00

    # extract morning sample column
    df_morning_sample = resampled_df.loc[resampled_df.hour==10,:].rename(columns={"COVID_copies/l": "COVID_morning_sample", "PMMoV_copies/l": "PMMoV_morning_sample"}).loc[:,  ["time", "time_in_days", "sampling_point", "memilio_id", "COVID_morning_sample", "PMMoV_morning_sample"]]
    # combine everything
    df_measurements = pd.merge(df, pd.merge(df_24h, df_morning_sample, on=["sampling_point", "memilio_id", "time", "time_in_days"], how="outer"), on=["sampling_point", "memilio_id", "time", "time_in_days"], how="outer")
    return df_measurements

In [ ]:
df_sampling = simulate_sampling_strategies(df, 3)

## Metrics

In [ ]:
sampling_point = "1"

In [ ]:
metric_24h_sample = df_sampling[["time_in_days", "sampling_point", "memilio_id", "COVID_24h_sample", "COVID_copies/l"]].copy()
metric_24h_sample = metric_24h_sample.loc[metric_24h_sample.sampling_point==sampling_point]

# take average across simulations
metric_24h_sample = metric_24h_sample.groupby("time_in_days")[["COVID_24h_sample", "COVID_copies/l"]].mean().sort_index()
# interpolate linearly between 24h samples
metric_24h_sample["COVID_24h_sample"] = metric_24h_sample["COVID_24h_sample"].interpolate(method="linear")
# calculate MAE
metric_24h_sample["MAE"] = np.abs(metric_24h_sample["COVID_24h_sample"] - metric_24h_sample["COVID_copies/l"])

In [ ]:
metric_24h_sample.describe()

In [ ]:
metric_morning_sample = df_sampling[["time_in_days", "sampling_point", "memilio_id", "COVID_morning_sample", "COVID_copies/l"]].copy()
metric_morning_sample = metric_morning_sample.loc[metric_morning_sample.sampling_point==sampling_point]

# take average across simulations
metric_morning_sample = metric_morning_sample.groupby("time_in_days")[["COVID_morning_sample", "COVID_copies/l"]].mean().sort_index()
# interpolate linearly between 24h samples
metric_morning_sample["COVID_morning_sample"] = metric_morning_sample["COVID_morning_sample"].interpolate(method="linear")
# calculate MAE
metric_morning_sample["MAE"] = np.abs(metric_morning_sample["COVID_morning_sample"] - metric_morning_sample["COVID_copies/l"])

In [ ]:
metric_morning_sample.describe()

In [ ]:
MAE_df = pd.DataFrame({"24h_sample": metric_24h_sample["MAE"], "morning_sample": metric_morning_sample["MAE"]})

In [ ]:
fig, ax = plt.subplots(figsize = (6,4), dpi=300)
sns.boxplot(data=MAE_df, linewidth=1.5)
ax.set_ylabel(f"absolute error")
ax.set_xticklabels(["24-hour\ncompound sample", "daily\ngrab sample"], size=15)
ax.tick_params(axis='y')
plt.tight_layout()
plt.savefig(os.path.join(plot_path, f"{file_name_prefix}_station_{sampling_point}_error.png"))

## Visualizations

In [ ]:
# from wide to long
df_sampling = pd.melt(df_sampling, id_vars=["time_in_days", "sampling_point", "memilio_id"], value_vars=["COVID_24h_sample", "COVID_morning_sample", "COVID_copies/l"]).dropna()

In [ ]:
df_sampling.head()

In [ ]:
def plot_sampling_strategies(df_sampling, station, save_fig=False):
    df_curr = df_sampling.loc[df_sampling.sampling_point==station, :]

    # Use Seaborn's default 95% confidence interval with error bars
    fig, ax = plt.subplots(figsize=(9, 4), dpi=300) 
    sns.lineplot(df_curr.loc[df_curr.variable == "COVID_copies/l", :], ax=ax, x="time_in_days", y="value", color="#2ca02c", label="reference", legend=False, alpha=0.2)

    ax2 = ax.twinx()
    sns.lineplot(df_curr.loc[df_curr.variable == "COVID_morning_sample", :], ax=ax2, x="time_in_days", y="value", errorbar=None, color="#ff7f0e", marker="o", label="daily grab sample")
    sns.lineplot(df_curr.loc[df_curr.variable == "COVID_24h_sample", :], ax=ax2, x="time_in_days", y="value", errorbar=None, color="steelblue", marker="o", label="24-hour compound sample")

    # Calculate and add 95% CI as error bars
    ci_morning = df_curr[df_curr.variable == "COVID_morning_sample"].groupby("time_in_days")['value'].agg(['mean', 'sem'])
    ci_24h = df_curr[df_curr.variable == "COVID_24h_sample"].groupby("time_in_days")['value'].agg(['mean', 'sem'])

    ax2.errorbar(ci_morning.index, ci_morning['mean'], yerr=1.96 * ci_morning['sem'], fmt='o', color="#ff7f0e", capsize=4)
    ax2.errorbar(ci_24h.index, ci_24h['mean'], yerr=1.96 * ci_24h['sem'], fmt='o', color="steelblue", capsize=4)

    ax.set_ylabel('virus levels [copies/l]')
    ax.set_xticks([0, 2, 4, 6, 8, 10, 12, 14])
    ax.set_xticklabels(['0', '2', '4', '6', '8', '10', '12', '14'])
    ax.set_xlabel("simulation time [days]")

    ax2.set_ylabel(None)
    ax2.set_yticks([])

    ax.set(ylim=(0, 500), xlim=(-0.5, 14.5))
    ax2.set(ylim=(0, 500), xlim=(-0.5, 14.5))

    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2)

    if save_fig:
        plt.tight_layout()
        plt.savefig(os.path.join(plot_path, f"{file_name_prefix}_station_{station}.png"))
        plt.close(fig)
    else:
        plt.close(fig)
        return fig

In [ ]:
plot_sampling_strategies(df_sampling, "16", save_fig=False)

In [ ]:
for station in ["1", "11", "16"]: # ["1", "2", "8", "16"]: # df_sampling.sampling_point.unique():
    plot_sampling_strategies(df_sampling, station, save_fig=True)